## This Notebook is for a Capstone

This will be the primary use of said notebook. Please be aware

In [1]:
!conda install -c conda-forge folium=0.5.0

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [47]:
import pandas as pd
import numpy as np
import bs4
from bs4 import BeautifulSoup
import requests
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors
from pandas.io.json import json_normalize
import matplotlib.colors as colors

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Time to Segment and Classify Toronto

### First
Scrape the Postal Code Data from the Wiki    
And throw it into a dataframe

In [4]:
#Puts Wikiscrape into DF
pc_tor = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(pc_tor.content,'lxml')
codes = soup.find_all('table')[0]
zips_df = pd.read_html(str(codes), header = 0)
zips_df= zips_df[0]

### Second
Remove not assigned Boroughs

In [5]:
zips_df= zips_df.where(zips_df['Borough']!= 'Not assigned').dropna()

### Third
Group by Postal Code and make the data neat

In [6]:
def bump_set_spike(ball):
    ball = set(ball)
    point = ''
    for play in range(len(ball)):
        if play != 0: point += ', '
        point += ball.pop()
    return point
print(bump_set_spike(['1','2','3']))

3, 2, 1


In [7]:
Gzips = zips_df.groupby(zips_df['Postcode'], as_index=True)

In [8]:
b_zips = Gzips['Borough'].apply(lambda x:bump_set_spike(x)).to_frame()
n_zips = Gzips['Neighbourhood'].apply(lambda x:bump_set_spike(x)).to_frame()
j_zips = b_zips.join(n_zips)

### Finally
Take care  of Not assigned Neighbourhood(s)

In [9]:
aHold = j_zips[j_zips.Neighbourhood.str.contains('Not assigned')]
aHold

,Borough,Neighbourhood
Postcode,,
M7A,Queen's Park,Not assigned


In [10]:
j_zips.loc[aHold.index[0],'Neighbourhood'] = aHold.Borough[0]

In [11]:
j_zips.shape

(103, 2)

# Geocoder Section
Geocoding the Post codes

I Don't wish to put in an API key so I will just pull .CSV

In [12]:
!wget -O postcode_data.csv http://cocl.us/Geospatial_data

--2019-06-10 01:30:51--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-06-10 01:30:51--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-10 01:30:52--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-10 01:30:52--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

### 

In [13]:
geopost = pd.read_csv('postcode_data.csv')
geopost.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
post_lola = j_zips.join(geopost.set_index('Postal Code'))
post_lola.head(10)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside",43.716316,-79.239476


# Foursquare API and Dataframe Organization

In [16]:
###FourSquare API
CLIENT_ID = 'P2Q0JKUA4CZXVHNS2VDQFM11K3DM1VSK2AHCGJPRK0WRGOEW' # your Foursquare ID
CLIENT_SECRET = 'UYFJOLE1LUZMB1VKQWAWFY0MOHSHXJ3NACX3K1IGAMOVOFWO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [17]:
###pulls category type
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
###cleans up the individual requests FOR USE IN four_pull
def ven_jdb(rawapi):
    venues = rawapi['response']['groups'][0]['items']
    
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    return (nearby_venues)

In [19]:
###combines Api Call and Pulling Relavent Data into a Dictionary
def four_pull(frame):
    postal_ven = {}
    for row in range(frame.shape[0]):
        llb = frame.iloc[row]
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius=2000&limit=100'.format(CLIENT_ID, CLIENT_SECRET, llb[2], llb[3], VERSION)
        fs_return = requests.get(url).json()
        try:
            postal_ven[frame.index[row]] = ven_jdb(fs_return)
        except Exception as e:
            print('Issue pulling Data from ' + row+': '  + e)
    return postal_ven

In [20]:
###take Venue data as Dictionary and turns it into ratio by Venue Type and return as Dictionary
def ratio_ven(post_dic):
    key_frame = {}
    for key in post_dic.keys():
        holden = post_dic[key].groupby(['categories']).count()
        holden = holden.T
        holden = holden/((post_dic[key].shape[0]))
        key_frame[key] = holden
        
    
    return key_frame

In [21]:
### Put it all together
api_pull = four_pull(post_lola)
ratio_frame = ratio_ven(api_pull)

In [60]:
'''Commented Out because is manual verification

###Verify that all post codes add up to 1.0 
for key in ratio_frame.keys():
    print(key)
    print(ratio_frame[key].iloc[1].sum())
    
'''

'Commented Out because is manual verification\n\n###Verify that all post codes add up to 1.0 \nfor key in ratio_frame.keys():\n    print(key)\n    print(ratio_frame[key].iloc[1].sum())\n    \n'

In [23]:
### Put it all in a Dataframe
firstpass = True
for key in ratio_frame.keys():
    
    dropper = ratio_frame[key].drop(index = ['lat', 'lng'])
    dropper.index = [key]
    if firstpass == True:
        full_counter = dropper
        firstpass = False
    else:
        full_counter=full_counter.append(dropper)
full_counter = full_counter.fillna(0)

In [24]:
### For use as Column names
col_ven = ['','Second ', 'Third ', 'Fourth ', 'Fifth ', 'Sixth ', 'Seventh ', 'Eighth ', 'Ninth ', 'Tenth ']
for top in range(len(col_ven)):
    col_ven[top]+= 'Most Popular Venue Type'

In [25]:
### Make a list of the data for Top 10 Venue types
tops_list = []
for post in full_counter.index:
    x = full_counter.loc[post]
    x = x.sort_values( ascending = False).head(10)
    tops_list.append(([x.name]+list(x.index)))

In [26]:
### 
tops_df = pd.DataFrame(data = tops_list, columns = ['Postcode']+col_ven)
tops_df = tops_df.set_index(['Postcode'])

In [41]:
### Run Clustering
kclusters = 5

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(full_counter.sort_index())


In [44]:
###Sort the Postcodes to match labels
label_lola = post_lola.sort_index()
label_lola.insert(2, 'Cluster Labels', kmeans.labels_)

In [76]:
### create map, markers and labels
map_malf = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(label_lola['Latitude'], label_lola['Longitude'], label_lola['Neighbourhood'], label_lola['Cluster Labels']):
    label = folium.Popup(str(poi) + ' | Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_malf)
       
map_malf

# Analyze and Name Clusters

In [62]:
### Join Dataframs
tops_lola = label_lola.join(tops_df)

In [69]:
clust0 = tops_lola.loc[tops_lola['Cluster Labels'] == 0]
faster_food = clust0.drop(columns=['Neighbourhood', 'Longitude', 'Latitude'])
faster_food

,Borough,Cluster Labels,Most Popular Venue Type,Second Most Popular Venue Type,Third Most Popular Venue Type,Fourth Most Popular Venue Type,Fifth Most Popular Venue Type,Sixth Most Popular Venue Type,Seventh Most Popular Venue Type,Eighth Most Popular Venue Type,Ninth Most Popular Venue Type,Tenth Most Popular Venue Type
Postcode,,,,,,,,,,,,
M1B,Scarborough,0,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Zoo,Athletics & Sports,Liquor Store,Paper / Office Supplies Store,Grocery Store,Gift Shop,Caribbean Restaurant
M1C,Scarborough,0,Pizza Place,Coffee Shop,Breakfast Spot,Pet Store,Sandwich Place,Ice Cream Shop,Food & Drink Shop,Bar,Liquor Store,Beer Store
M1E,Scarborough,0,Pizza Place,Breakfast Spot,Park,Fast Food Restaurant,Coffee Shop,Pharmacy,Fried Chicken Joint,Bank,Bar,Liquor Store
M1G,Scarborough,0,Coffee Shop,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Furniture / Home Store,Supermarket,Beer Store,Indian Restaurant,Discount Store,Pharmacy
M1J,Scarborough,0,Fast Food Restaurant,Coffee Shop,Pizza Place,Pharmacy,Sandwich Place,Grocery Store,Liquor Store,Beer Store,Big Box Store,Discount Store
M1K,Scarborough,0,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Bank,Sandwich Place,Discount Store,Beer Store,Laser Tag
M1L,Scarborough,0,Coffee Shop,Fast Food Restaurant,Sandwich Place,Shoe Store,Burger Joint,Pizza Place,Clothing Store,Dessert Shop,Shopping Plaza,Burrito Place
M1M,Scarborough,0,Fast Food Restaurant,Park,Harbor / Marina,Coffee Shop,Beach,Sandwich Place,Grocery Store,Pizza Place,Pharmacy,Pub
M1P,Scarborough,0,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pizza Place,Pet Store,Wings Joint,Pharmacy,Breakfast Spot,Grocery Store,Bookstore


In [71]:
clust1 = tops_lola.loc[tops_lola['Cluster Labels'] == 1]
shopping_asia = clust1.drop(columns=['Neighbourhood', 'Longitude', 'Latitude'])
shopping_asia

,Borough,Cluster Labels,Most Popular Venue Type,Second Most Popular Venue Type,Third Most Popular Venue Type,Fourth Most Popular Venue Type,Fifth Most Popular Venue Type,Sixth Most Popular Venue Type,Seventh Most Popular Venue Type,Eighth Most Popular Venue Type,Ninth Most Popular Venue Type,Tenth Most Popular Venue Type
Postcode,,,,,,,,,,,,
M1H,Scarborough,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Gym,Wings Joint,Pharmacy,Sandwich Place,Restaurant,Sporting Goods Shop,Indian Restaurant
M1N,Scarborough,1,Coffee Shop,Bank,Grocery Store,Park,Pizza Place,Fish & Chips Shop,Pharmacy,Fast Food Restaurant,Asian Restaurant,Beer Store
M1R,Scarborough,1,Coffee Shop,Middle Eastern Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Burger Joint,Sandwich Place,Restaurant,Vietnamese Restaurant,Chinese Restaurant
M1S,Scarborough,1,Chinese Restaurant,Coffee Shop,Pharmacy,Cantonese Restaurant,Sandwich Place,Restaurant,Indian Restaurant,Shopping Mall,Fast Food Restaurant,Sushi Restaurant
M1V,Scarborough,1,Chinese Restaurant,Coffee Shop,Park,Pizza Place,Bakery,Indian Restaurant,Japanese Restaurant,Dessert Shop,Noodle House,Vietnamese Restaurant
M1W,Scarborough,1,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Bakery,Pharmacy,Japanese Restaurant,Sandwich Place,Intersection,Breakfast Spot,Athletics & Sports
M2H,North York,1,Coffee Shop,Park,Chinese Restaurant,Sandwich Place,Bank,Bakery,Asian Restaurant,Pizza Place,Japanese Restaurant,Pharmacy
M2J,North York,1,Coffee Shop,Clothing Store,Fast Food Restaurant,Middle Eastern Restaurant,Pharmacy,Sandwich Place,Pizza Place,Japanese Restaurant,Park,Chinese Restaurant
M2K,North York,1,Chinese Restaurant,Park,Coffee Shop,Pharmacy,Bank,Café,Grocery Store,Convenience Store,Bakery,Szechuan Restaurant


In [72]:
clust2 = tops_lola.loc[tops_lola['Cluster Labels'] == 2]
mediterranean_easy = clust2.drop(columns=['Neighbourhood', 'Longitude', 'Latitude'])
mediterranean_easy

,Borough,Cluster Labels,Most Popular Venue Type,Second Most Popular Venue Type,Third Most Popular Venue Type,Fourth Most Popular Venue Type,Fifth Most Popular Venue Type,Sixth Most Popular Venue Type,Seventh Most Popular Venue Type,Eighth Most Popular Venue Type,Ninth Most Popular Venue Type,Tenth Most Popular Venue Type
Postcode,,,,,,,,,,,,
M4C,East York,2,Pizza Place,Coffee Shop,Park,Café,Gastropub,Ice Cream Shop,Pharmacy,Ethiopian Restaurant,Thai Restaurant,Fast Food Restaurant
M4E,East Toronto,2,Coffee Shop,Pub,Beach,Breakfast Spot,Bar,Bakery,Thai Restaurant,Japanese Restaurant,Park,Café
M4J,East York,2,Greek Restaurant,Café,Coffee Shop,Bakery,Pizza Place,Pub,Gastropub,American Restaurant,Park,Brewery
M4K,East Toronto,2,Greek Restaurant,Café,Park,Vietnamese Restaurant,Bakery,Ice Cream Shop,American Restaurant,Italian Restaurant,Pub,Coffee Shop
M4L,East Toronto,2,Park,Café,Coffee Shop,Brewery,Bar,Beach,Bakery,Italian Restaurant,American Restaurant,BBQ Joint
M4M,East Toronto,2,Coffee Shop,Park,Café,Brewery,Bakery,Vietnamese Restaurant,Bar,French Restaurant,Restaurant,Pizza Place
M4T,Central Toronto,2,Italian Restaurant,Park,Café,Sushi Restaurant,Bakery,Coffee Shop,Dessert Shop,Indian Restaurant,Grocery Store,Restaurant
M6C,York,2,Coffee Shop,Italian Restaurant,Bank,Pizza Place,Mexican Restaurant,Café,Bakery,Thai Restaurant,Bagel Shop,Ice Cream Shop
M6E,York,2,Italian Restaurant,Sandwich Place,Coffee Shop,Bakery,Bank,Furniture / Home Store,Breakfast Spot,Pizza Place,Mexican Restaurant,Grocery Store


In [73]:
clust3 = tops_lola.loc[tops_lola['Cluster Labels'] == 3]
downtown_variety = clust3.drop(columns=['Neighbourhood', 'Longitude', 'Latitude'])
downtown_variety

,Borough,Cluster Labels,Most Popular Venue Type,Second Most Popular Venue Type,Third Most Popular Venue Type,Fourth Most Popular Venue Type,Fifth Most Popular Venue Type,Sixth Most Popular Venue Type,Seventh Most Popular Venue Type,Eighth Most Popular Venue Type,Ninth Most Popular Venue Type,Tenth Most Popular Venue Type
Postcode,,,,,,,,,,,,
M2N,North York,3,Korean Restaurant,Bubble Tea Shop,Burrito Place,Café,Sushi Restaurant,Supermarket,Liquor Store,Theater,Thai Restaurant,Bakery
M4P,Central Toronto,3,Coffee Shop,Café,Italian Restaurant,Bakery,Indian Restaurant,Park,Sushi Restaurant,Japanese Restaurant,Gym,Yoga Studio
M4R,Central Toronto,3,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Bakery,Japanese Restaurant,Gastropub,Burger Joint,Wings Joint,Tea Room
M4S,Central Toronto,3,Coffee Shop,Café,Bakery,Italian Restaurant,Sushi Restaurant,Park,Yoga Studio,Gastropub,Indian Restaurant,Gym
M4V,Central Toronto,3,Italian Restaurant,Café,Coffee Shop,Park,Spa,Grocery Store,Hotel,French Restaurant,Middle Eastern Restaurant,Ice Cream Shop
M4W,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Park,French Restaurant,Japanese Restaurant,Spa,Grocery Store,Café,Restaurant,Gastropub
M4X,Downtown Toronto,3,Coffee Shop,Park,Café,Gastropub,Restaurant,Thai Restaurant,Ramen Restaurant,Japanese Restaurant,Italian Restaurant,Dance Studio
M4Y,Downtown Toronto,3,Park,Coffee Shop,Gastropub,Italian Restaurant,Japanese Restaurant,Diner,Pizza Place,Ramen Restaurant,Hotel,Comic Shop
M5A,Downtown Toronto,3,Coffee Shop,Café,Park,Restaurant,Thai Restaurant,Italian Restaurant,Bakery,Gastropub,Farmers Market,Japanese Restaurant


In [74]:
clust4 = tops_lola.loc[tops_lola['Cluster Labels'] == 4]
parks_n_rec = clust4.drop(columns=['Neighbourhood', 'Longitude', 'Latitude'])
parks_n_rec

,Borough,Cluster Labels,Most Popular Venue Type,Second Most Popular Venue Type,Third Most Popular Venue Type,Fourth Most Popular Venue Type,Fifth Most Popular Venue Type,Sixth Most Popular Venue Type,Seventh Most Popular Venue Type,Eighth Most Popular Venue Type,Ninth Most Popular Venue Type,Tenth Most Popular Venue Type
Postcode,,,,,,,,,,,,
M1X,Scarborough,4,Grocery Store,Golf Course,Sculpture Garden,Trail,Farm,Playground,Electronics Store,Dongbei Restaurant,Donut Shop,Dumpling Restaurant


# Quick Rundown of Analysis

### Most Locations have high rates of coffee shops and cafes


### Cluster 0:    
Has high amounts of Fast food and discount stores.

### Cluster 1:
Most incidents of asian cuisine. Has many shopping stores as well.

### Cluster 2:
Plenty of greek and italian food. Also many parks.

### Cluster 3:
It's downtown and has the most variety of food. 

### Cluster 4